# Import libraries

In [ ]:
import os
import sys
import json
import csv
import time
import copy
from IPython import display

# numeric, scientific
from scipy import linalg as spla
from scipy.integrate import simps
import numpy as np

# plotting
%matplotlib inline
import matplotlib
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import Kitaev Honeycomb package
import kithcmb
from kithcmb import ThermalGradient as gradvs
from kithcmb import VortexSectorThermalFermions as nongradvs

# import jsci, CT's enhanced json stream write package
import jsci
from jsci import WriteStream as jsciwrite
from jsci import Coding as jscicoding

In [ ]:
plt.style.use('prettyfigs')

# Set system parameters and initialise 

In [ ]:
L = 8
J = 1.
kappa = 0.1
T = 2.0
dt = 0.1
max_t = 2.
times = np.arange(0,max_t,dt)

In [ ]:
non_grad_sys = nongradvs.VortexSectorThermalFermions(L,J,kappa)
non_grad_eigvecs = non_grad_sys._get_eigenvectors()

In [ ]:
# compute the density matrix of the non-gradient state in basis of 
# the thermal gradient eigenstates
ferm_occs = non_grad_sys.get_fermionic_expectation_values(1./T)
combined_fermionic_expectation_values = np.empty( 2 * L**2 )
for m in range(L**2):
    combined_fermionic_expectation_values[m] = np.exp( ferm_occs[2][m] )
    combined_fermionic_expectation_values[2 * L**2 - 1 - m] = np.exp( ferm_occs[1][m] )
combined_fermionic_expectation_values = np.diag( combined_fermionic_expectation_values )
dens_mat = combined_fermionic_expectation_values

In [ ]:
# arrays for storing the total currents in the x- and z- dirn as a function of time
xtot = []
ztot = []

# evolve the density matrix in time
for t in np.arange(0,max_t,dt):
    start_time = time.time()
    U = np.diag( np.exp(-1j * non_grad_sys.spectrum * t) )
    dens_mat_at_t = np.dot( U, dens_mat )
    dens_mat_at_t = np.dot( dens_mat_at_t, np.matrix(U).getH() )

    # multiply the fermionic expectation matrix with the eigenvectors to rotate to the
    # Majorana basis
    correl_matrix = np.dot( np.matrix(non_grad_eigvecs), np.matrix(dens_mat_at_t) )
    correl_matrix = np.dot( correl_matrix, np.matrix(non_grad_eigvecs).getH() )

    # there can be an arbitrary & non-physical real component of correl_matrix, we throw that away here
    # (the real diagonal does have some meaning so we keep it)
    diag_correl = np.diag(correl_matrix)
    correl_matrix = 1j * np.imag(correl_matrix) + np.diag(diag_correl)

    # cache the correlation matrix
    # the factor of two corrects for the \hat{a} fermions being prop to 1/2 (sum of c's) rather than 1/sqrt(2)
    # this is equivalent to the statement that <c_i c_j> = 2 P_ij
    correl_matrix = 2. * np.array(correl_matrix)

    # get the currents
    currents = non_grad_sys._compute_thermal_current_matrix(correl_matrix)

    # output the currents
    # file_name = output_name+"{0:.2g}".format(t)
    # np.savetxt( file_name+".csv",np.real(currents), delimiter="," )
    
    display.clear_output(wait=True)
    fig,(ax1,ax2) = plt.subplots( 1,2,figsize=(16,8) )
    
    # get the net currents
    net_current,temp_currents = non_grad_sys._sum_curr_in_x_dirn(currents)
    net_current = np.sum( net_current, axis=0 )
    ax1.set_title('x-dirn, sum : '+str(np.sum(np.real(net_current))),fontsize=20)
    ax1.plot( np.real(net_current) )
    ax1.set_xlabel(r'$z$')
    xtot.append(np.sum(np.real(net_current)))
    #
    net_current,temp_currents = non_grad_sys._sum_curr_in_y_dirn(currents)
    net_current = np.sum( net_current, axis=1 )
    ax2.set_title('z-dirn, sum : '+str(np.sum(np.real(net_current))),fontsize=20)
    ax2.plot( np.real(net_current) )
    ax2.set_xlabel(r'$x$')
    ztot.append(np.sum(np.real(net_current)))
    
    plt.show()
    print 't : ',t
    
    print('took '+str(time.time()-start_time)+' seconds')

In [ ]:
fig,ax = plt.subplots()
plt.plot( times,xtot, times,ztot )
#ax.set_xlim([-20,20])

In [ ]:
fig,ax = plt.subplots()
plt.plot( times,xtot, 'x', times,ztot, 'x' )